In [ ]:
# !pip install ogame

In [ ]:
# https://github.com/alaingilbert/pyogame/wiki/Code-Snippets

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../pyogame'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ogame import OGame, constants
import time

In [2]:
o = OGame(
    universe="Ymir",
    # universe='Volans',
    username="samchakerian@gmail.com",
    password="$R3e@W1q1q"
)

In [3]:
# # time 
# sp = 5
# mm = 3
# cm = 3
# dm = 15

In [4]:
# income
# mm1 : +504
# mm2: ???
# mm3: +969
# cm : +120
# cm2: +414

In [5]:
# make class with starting res, build times, income

# should initiate building, unless you need to wait for resources

# in that case, should calculate wait time



In [6]:
# id = o.planet_ids()[0]
# o.build(constants.buildings.solar_plant, id)
# time.sleep(sp)
# o.build(constants.buildings.metal_mine, id)
# time.sleep(mm)
# o.build(constants.buildings.metal_mine, id)
# time.sleep(mm*2)
# o.build(constants.buildings.solar_plant, id)
# time.sleep(sp*2)
# o.build(constants.buildings.metal_mine, id)
# time.sleep(mm*3)
# o.build(constants.buildings.metal_mine, id)
# time.sleep(mm*4)
# o.build(constants.buildings.solar_plant, id)
# time.sleep(sp*3)
# o.build(constants.buildings.crystal_mine, id)
# time.sleep(cm)
# o.build(constants.buildings.solar_plant, id)
# time.sleep(sp*4)
# o.build(constants.buildings.metal_mine, id)
# time.sleep(mm*5)
# o.build(constants.buildings.crystal_mine, id)
# time.sleep(cm*2)
# o.build(constants.buildings.crystal_mine, id)
# time.sleep(cm*3)
# o.build(constants.buildings.solar_plant, id)
# time.sleep(sp*5)
# o.build(constants.buildings.deuterium_mine, id)
# time.sleep(dm)

In [7]:
# # https://ogame.fandom.com/wiki/Quick_Start_Guide

# Step // Building/Tech // Building Tech Level // notes // (Sam time)

# 1 // Solar Plant // 1 // 5s
# 2 // Metal mine // 1 // 3s
# 3 // Metal Mine // 2 // 
# 4 // Solar Plant // 2 // 
# 5 // Metal Mine // 3 // +8 Energy
# 6 // Metal Mine // 4 // - 11 Energy but improvement in production
# 7 // Solar Plant // 3
# 8 // Crystal Mine // 1 // +9 energy. Do not upgrade. Stick to the guide
# 9 // Solar Plant // 4
# 10 // Metal Mine // 5 // You can also switch steps 10 and 11 if you're doing the tutorial to get free rss
# 11 // Crystal Mine // 2 // +11 energy
# 12 // Crystal Mine // 3 // -4 energy but do upgrade because improvement in production. 
# Metal prod decreases with 9 metal/h no biggy
# 13 // Solar Plant // 5 // Make sure to build this as soon as you can to reduce unnecessary production loss

# 14 // Deuterium Synthesizer // 1 // Energy +18

# 15 // Crystal Mine // 4 // Energy -1 Upgrade in production so build it.

# 16 // Solar Plant // 6

# 17 // Metal Mine // 6

# 18 // Metal Mine // 7 // -6 but improvement (produce at 90%)

# 19 // Solar Plant // 7 // put Metal Mine back to 100% after done

# 20 // Crystal Mine // 5

# 21 // Deuterium Synthesizer // 2

# 22 // Solar Plant // 8

# 23 // Deuterium Synthesizer // 3

# 24 // Deuterium Synthesizer // 4

# 25 // Solar Plant // 9

# 26 // Deuterium Synthesizer // 5

# 27 // Robotics Factory // 1 // Robo 1 is the first building that costs deuterium. 
# Continue with the list in the given order as much as you can. If you dont have deuterium left, 
# then just continue with non deuterium costing buildings.

# 28 // Robotics Factory // 2

# 29 // Research Lab // 1

# 30 // Shipyard // 1

# 31 // Crystal Mine // 6

# 32 // Shipyard // 2

# 33 // Solar Plant // 10

# 34 // Deuterium Synthesizer // 6

# 35 // Metal Mine // 8

# 36 // Energy Research // 1

# 37 // Combustion Drive // 1 // If you have enough resources

# 38 // Solar Plant // 11 // Whenever you can research combustion drive 2, research it.

# 39 // Crystal Mine // 7 // Whenever you can build Shipyard 3, build it

# 40 // Metal Mine // 9



In [8]:
id = o.planet_ids()[0]
supply = o.supply(id)
res = o.resources(id)

In [9]:
# check bldg construction
{
    'sp': supply.solar_plant.in_construction,
    'mm': supply.metal_mine.in_construction,
    'cm': supply.crystal_mine.in_construction,
    'dm': supply.deuterium_mine.in_construction
}

{'sp': False, 'mm': False, 'cm': False, 'dm': False}

In [10]:
# is possible
{
    'sp': supply.solar_plant.is_possible,
    'mm': supply.metal_mine.is_possible,
    'cm': supply.crystal_mine.is_possible,
    'dm': supply.deuterium_mine.is_possible
}

{'sp': True, 'mm': True, 'cm': True, 'dm': True}

In [11]:
# check resources
{
    'sp': res.energy,
    'mm': res.metal,
    'cm': res.crystal,
    'dm': res.deuterium
}

{'sp': 418, 'mm': 75000, 'cm': 40000, 'dm': 39136}

In [12]:
# check levels
{
    'sp': supply.solar_plant.level,
    'mm': supply.metal_mine.level,
    'cm': supply.crystal_mine.level,
    'dm': supply.deuterium_mine.level
}

{'sp': 14, 'mm': 10, 'cm': 8, 'dm': 8}

In [13]:
# o.is_logged_in()

# o.relogin()

# o.logout()

In [14]:
# # keep_going decorator
# @empire.keep_going
# def run():
#     while True:
#         print(empire.attacked())
#         time.sleep(1)

In [15]:
# attack inactives

# # Send a Spyprobe to a random Player
# send_fleet = False
# while not send_fleet:
#     for planet in empire.galaxy(coordinates(randint(1,6), randint(1, 499))):
#         if empire.send_fleet(mission.spy, id, planet.position, [ships.espionage_probe(1)]):
#             send_fleet = True
#             break

# # Wait till Spyprobe arrives
# for fleet in empire.fleet():
#     if fleet.mission == mission.spy:
#         while datetime.now() < fleet.arrival:
#             continue

# # Print out Spyreport
# for spyreport in o.spyreports():
#     print(spyreport.list)

# for planet in empire.galaxy(coordinates(1, 1)):
#     if status.inactive in planet.status and status.vacation not in planet.status:
#         empire.send_fleet(mission=mission.attack,
#                           id=empire.id_by_planet_name('MAIN_PLANET'),
#                           where=coordinates(1, 1, planet.position),
#                           ships=[ships.large_transporter(25), ships.cruiser(1)])

In [16]:
o.planet_ids()[0]

33624031

In [17]:
o.celestial_coordinates(33624031)

[1, 447, 10, 1]

In [98]:
# for planet in o.galaxy(constants.coordinates(1, 447)):
#     print(planet.status)
    
for galaxy in range(441, 455):
    for planet in o.galaxy(constants.coordinates(1, galaxy)):
        # print(galaxy, planet.status, planet.name, planet.rank)
        if 'inactive' in planet.status and 'vacation' not in planet.status:
            print(galaxy, planet.status, planet.name, planet.rank)
            print(planet.position)
#             o.send_fleet(
#                 constants.mission.spy, 
#                 o.planet_ids()[0], 
#                 planet.position, 
#                 [constants.ships.espionage_probe(1)]
#             )

441 ['inactive'] Homeworld 1459
[1, 441, 10, 1]
444 ['inactive'] Homeworld 1436
[1, 444, 8, 1]
444 ['inactive'] Howak 1121
[1, 444, 12, 1]
445 ['inactive'] Homeworld 1347
[1, 445, 4, 1]
445 ['inactive'] Homeworld 1136
[1, 445, 6, 1]
448 ['inactive'] Homeworld 1575
[1, 448, 10, 1]
449 ['inactive'] Homeworld 1631
[1, 449, 12, 1]
450 ['inactive'] Beer 566
[1, 450, 8, 1]
451 ['inactive'] Homeworld 1527
[1, 451, 6, 1]
451 ['inactive'] Homeworld 1528
[1, 451, 12, 1]
452 ['inactive'] Homeworld 1257
[1, 452, 6, 1]
453 ['inactive'] Homeworld 1109
[1, 453, 4, 1]


In [97]:
planet.position

[1, 444, 12, 1]

In [ ]:
# response = o.session.get(o.index_php + 'page=ingame&component=fleetdispatch&cp={}'.format(33624031))

In [19]:
# response.text

In [20]:
import re

In [21]:
# send_fleet_token = re.search(
#     # 'var fleetSendingToken = "(.*)"',
#     'var token = "(.*)"',
#     response.text
# ).group(1)
# form_data = {'token': send_fleet_token}

In [22]:
# form_data

# ships = [constants.ships.espionage_probe(1)]

# send_fleet_token = 'b13bf8f7e593865de07d7e057f8b8e3c' # from inspect

# where = [1, 440, 8, 1]
# resources = (0, 0, 0)
# mission = constants.mission.spy
# speed = 10
# holdingtime = 0

# form_data = {'token': send_fleet_token}

# for ship in ships:
#     ship_type = 'am{}'.format(ship[0])
#     form_data.update({ship_type: ship[1]})

# form_data.update(
#     {
#         'galaxy': where[0],
#         'system': where[1],
#         'position': where[2],
#         'type': where[3],
#         'metal': resources[0],
#         'crystal': resources[1],
#         'deuterium': resources[2],
#         'prioMetal': 1,
#         'prioCrystal': 2,
#         'prioDeuterium': 3,
#         'mission': mission,
#         'speed': speed,
#         'retreatAfterDefenderRetreat': 0,
#         'union': 0,
#         'holdingtime': holdingtime
#     }
# )

# # response = self.session.post(
# #     url=self.index_php + 'page=ingame&component=fleetdispatch'
# #         '&action=sendFleet&ajax=1&asJson=1',
# #     data=form_data,
# #     headers={'X-Requested-With': 'XMLHttpRequest'}
# # ).json()
# # return response['success']

In [23]:
# form_data

# response = o.session.post(
#     url=o.index_php + 'page=ingame&component=fleetdispatch&action=sendFleet&ajax=1&asJson=1',
#     data=form_data,
#     headers={'X-Requested-With': 'XMLHttpRequest'}
# )

# response.json()

In [24]:
# id = o.planet_ids()[0]

In [25]:
# from ogame.constants import coordinates, mission, speed, fleet

In [26]:
# fleet

In [27]:
# # from ogame.constants import coordinates, mission, speed, fleet
# o.send_fleet(mission=mission.expedition,
#                   id=id,
#                   where=coordinates(1, 440, 8),
#                   ships=fleet(espionage_probe=1),
#                   resources=[0, 0, 0],  # optional default no resources
#                   speed=speed.max,      # optional default speed.max
#                   holdingtime=2)        # optional default 0 will be needed by expeditions

In [28]:
# # o.spyreports()
# o.spyreports()[0].fright

In [29]:
from bs4 import BeautifulSoup

def BeautifulSoup4(response):
    parsed = BeautifulSoup(response, features="html5lib")

    def find_partial(**kwargs):
        for key, value in kwargs.items():
            kwargs[key] = re.compile(value)
        return parsed.find(**kwargs)

    def find_all_partial(**kwargs):
        for key, value in kwargs.items():
            kwargs[key] = re.compile(value)
        return parsed.find_all(**kwargs)

    parsed.find_partial = find_partial
    parsed.find_all_partial = find_all_partial
    return parsed

In [30]:
# o.planet_ids()

In [31]:
response = o.session.get(
    url=o.index_php,
    params={'page': 'messages',
            'tab': 20,
            'ajax': 1}
).text
bs4 = BeautifulSoup4(response)
report_links = [
    link['href']
    for link in bs4.find_all_partial(href='page=messages&messageId')
]

In [32]:
report_links

['https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989723&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989715&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989712&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989705&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989698&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989696&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989694&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989686&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989680&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=98967

In [33]:
data = []

reports = []
for link in report_links:
    response = o.session.get(link).text
    bs4 = BeautifulSoup4(response)
#     technologys = [tech['class'][0] for tech in bs4.find_all('img')]
#     amounts = [
#         tech.parent.parent.find_all('span')[1].text
#         for tech in bs4.find_all('img')
#     ]
    data.append({'link': link, 'text': bs4})

In [34]:
len(data)

10

In [36]:
# data[0]['text'].find_all('span', class_="res_value")

# data[0]['text'].find('span', class_="msg_title new blue_txt").text

# [_.text for _ in data[0]['text'].find('ul', attrs={"data-type": "defense"}).find('li').find_all('span')]

'Espionage report from Homeworld [1:453:4]'

In [47]:
for report in data:
    rep_res = [_.text for _ in report['text'].find_all('span', class_="res_value")]
    rep_coord = report['text'].find('span', class_="msg_title new blue_txt").text
    try:
        rep_def = [_.text for _ in report['text'].find('ul', attrs={"data-type": "defense"}).find('li').find_all('span')]
    except:
        rep_def = None
    report['results'] = {
        'res': rep_res,
        'rep_coord': rep_coord,
        'rep_def': rep_def
    }

In [48]:
[_['results'] for _ in data]

[{'res': ['30.154', '24.781', '9.434', '272'],
  'rep_coord': 'Espionage report from Homeworld [1:453:4]',
  'rep_def': ['Rocket Launcher', '4']},
 {'res': ['12.682', '6.413', '4.501', '128'],
  'rep_coord': 'Espionage report from Homeworld [1:452:6]',
  'rep_def': None},
 {'res': ['10.000', '8.447', '0', '48'],
  'rep_coord': 'Espionage report from Homeworld [1:451:12]',
  'rep_def': None},
 {'res': ['6.559', '5.342', '0', '48'],
  'rep_coord': 'Espionage report from Homeworld [1:451:6]',
  'rep_def': None},
 {'res': ['133.119', '56.425', '25.532', '2.324', '4.200', '4.200'],
  'rep_coord': 'Espionage report from Beer [1:450:8]',
  'rep_def': []},
 {'res': ['10.000', '10.000', '0', '48'],
  'rep_coord': 'Espionage report from Homeworld [1:449:12]',
  'rep_def': None},
 {'res': ['10.000', '9.340', '3.124', '24'],
  'rep_coord': 'Espionage report from Homeworld [1:448:10]',
  'rep_def': None},
 {'res': ['10.000', '10.000', '10.000', '689'],
  'rep_coord': 'Espionage report from Homeworl

In [82]:
# find how many res you can get:

for _ in [_['results'] for _ in data]:
    if _['rep_def'] == None:
        res_clean = [int(__.replace('.', '')) for __ in _['res'][:3]]
        print(_['rep_coord'], sum(res_clean)/2)
        print(
            [int(_) for _ in re.search(r'[0-9:]{1,}', _['rep_coord']).group(0).split(":")],
        )

Espionage report from Homeworld [1:452:6] 11798.0
[1, 452, 6]
Espionage report from Homeworld [1:451:12] 9223.5
[1, 451, 12]
Espionage report from Homeworld [1:451:6] 5950.5
[1, 451, 6]
Espionage report from Homeworld [1:449:12] 10000.0
[1, 449, 12]
Espionage report from Homeworld [1:448:10] 11232.0
[1, 448, 10]
Espionage report from Homeworld [1:445:4] 15000.0
[1, 445, 4]
Espionage report from Howak [1:444:12] 35000.0
[1, 444, 12]


In [86]:
id

33624031

In [88]:
# get ships

ships = o.ships(id)

In [89]:
print(
    ships.espionage_probe.amount,
    ships.espionage_probe.is_possible,
    ships.espionage_probe.in_construction
)

print(
    ships.small_transporter.amount,
    ships.small_transporter.is_possible,
    ships.small_transporter.in_construction
)

print(
    ships.large_transporter.amount,
    ships.large_transporter.is_possible,
    ships.large_transporter.in_construction
)

10 True False


In [99]:
# deploy ships

for _ in [_['results'] for _ in data]:
    if _['rep_def'] == None:
        res_clean = [int(__.replace('.', '')) for __ in _['res'][:3]]
        print(
            _['rep_coord'], 
            sum(res_clean)/2, 
            [int(_) for _ in re.search(r'[0-9:]{1,}', _['rep_coord']).group(0).split(":")],
        )
        
        coords = [int(_) for _ in re.search(r'[0-9:]{1,}', _['rep_coord']).group(0).split(":")]
        
        o.send_fleet(
            constants.mission.attack,
            o.planet_ids()[0],
            (coords[0], coords[1], coords[2], 1),
            [constants.ships.small_transporter(2)]
        )

Espionage report from Homeworld [1:452:6] 11798.0 [1, 452, 6]
Espionage report from Homeworld [1:451:12] 9223.5 [1, 451, 12]
Espionage report from Homeworld [1:451:6] 5950.5 [1, 451, 6]
Espionage report from Homeworld [1:449:12] 10000.0 [1, 449, 12]
Espionage report from Homeworld [1:448:10] 11232.0 [1, 448, 10]
Espionage report from Homeworld [1:445:4] 15000.0 [1, 445, 4]
Espionage report from Howak [1:444:12] 35000.0 [1, 444, 12]


In [101]:
# o.send_fleet(
#     constants.mission.attack,
#     o.planet_ids()[0],
#     (coords[0], coords[1], coords[2], 1),
#     [constants.ships.large_transporter(1)]
# )